In [1]:
%load_ext autoreload
%autoreload 2
import util
import os, shutil
import yaml
import pandas as pd
import subprocess

In [2]:
# shift_by = 512
# new_shift_dir = 'chr8_shift_' + str(shift_by)
# new_unmap_file = '{}/sh{}_GRCh38_unmap.bed'.format(new_shift_dir, shift_by)
# util.shift_unmap(512, new_unmap_file)
# templ_yaml = 'chr8/template_config.yaml'
# new_yaml = new_shift_dir + '/template_config.yaml'

# with open(templ_yaml, 'r') as config_yaml:
#     config_dict = yaml.safe_load(config_yaml)
    
# config_dict['genomefile']['unmap'] = new_unmap_file

# with open(new_yaml, 'w') as config_yaml:
#     yaml.dump(config_dict, config_yaml)
# ! ./create_25cell_datasets.py chr8_shift_512
# shifted_dir = '/home/shush/profile/QuantPred/datasets/chr8_shift_{}/complete/random_chop/i_2048_w_1/'.format(shift_by)
# sts_shifted = util.load_stats(shifted_dir)
# shifted_testset = util.make_dataset(shifted_dir, 'test', sts_shifted, batch_size=64, shuffle=False, coords=True)
# model, b = read_model('1run', compile_model=True)
# bw_file = open_bw('1run/shifted/8_A549_pred_{}_raw.bw'.format(shift_by), 
#                   chrom_size_path="/home/shush/genomes/GRCh38_EBV.chrom.sizes.tsv")
# for C, x, y in shifted_testset:
#     C = [str(c).strip('b\'').strip('\'') for c in C.numpy()] # coordinates
#     P = model(x)
#     for i, pred in enumerate(P):
#         chrom, start, end = C[i].split('_') # get chr, start, end
#         start = int(start) # to feed into bw making function
#         # write to prediction bw file
#         bw_file.addEntries(chrom, start,
#             values=np.array(np.squeeze(pred[:,8]), dtype='float64'),
#             span=1, step=1)
# bw_file.close()

In [8]:
BASE_DIR = '15_IDR_test_sets_6K'
# per cell line
for i in range(15):
    # make new dir 
    cell_line_dir = os.path.join(BASE_DIR, 'cell_line_{}'.format(i))
    os.mkdir(cell_line_dir)
    # make cell line specific basset samplefile
    orig_basset_filepath = '/mnt/906427d6-fddf-41bf-9ec6-c3d0c37e766f/amber/ATAC/basset_sample_file.tsv'
    orig_basenji_filepath = '/mnt/906427d6-fddf-41bf-9ec6-c3d0c37e766f/amber/ATAC/basenji_sample_file.tsv'
    new_basset_filepath = os.path.join(cell_line_dir, 'basset_sample_file.tsv')
    new_basenji_filepath = os.path.join(cell_line_dir, 'basenji_sample_file.tsv')
    bed_line = pd.read_csv(orig_basset_filepath, sep='\t', header=None).iloc[[i]]
    bed_line.to_csv(new_basset_filepath, 
                    header=None, index=None, sep='\t')
    bw_line = pd.read_csv(orig_basenji_filepath, sep='\t').iloc[[i]]
    bw_line.to_csv(os.path.join(cell_line_dir, 'basenji_sample_file.tsv'), 
                     index=None, sep='\t')
    # scp template 
    blank_template = BASE_DIR + '/template_config.yaml'
    shutil.copy(blank_template, os.path.join(cell_line_dir, 'template_config.yaml'))
    cmd = './create_25cell_datasets.py {} {} {}'.format(cell_line_dir, new_basset_filepath, new_basenji_filepath)
#     cmd = './create_25cell_datasets.py cell_line_specific_test_sets/cell_line_0 cell_line_specific_test_sets/cell_line_0/basset_sample_file.tsv cell_line_specific_test_sets/cell_line_0/basenji_sample_file.tsv'
    process = subprocess.Popen(cmd, shell=True)
    output, error = process.communicate()